## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-26-10-40-54 +0000,nyt,Trump Administration Live Updates: President O...,https://www.nytimes.com/live/2025/10/25/us/tru...
1,2025-10-26-10-30-17 +0000,wapo,"Police arrest Louvre heist suspects, Paris pro...",https://www.washingtonpost.com/world/2025/10/2...
2,2025-10-26-10-28-16 +0000,bbc,Hurricane Melissa strengthens as it barrels to...,https://www.bbc.com/news/articles/ckgkqvxkzx0o...
3,2025-10-26-10-24-24 +0000,nyt,"Police Make Arrests in Louvre Robbery, Authori...",https://www.nytimes.com/2025/10/26/world/europ...
4,2025-10-26-10-17-13 +0000,nyt,Here’s the latest.,https://www.nytimes.com/2025/10/25/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2385/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,36
13,louvre,9
34,asia,8
164,house,8
465,he,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
80,2025-10-26-01-00-00 +0000,wsj,"“You have zero zoning conditions, you’re the p...",https://www.wsj.com/politics/policy/trump-whit...,76
50,2025-10-26-06-49-24 +0000,nypost,Trump signs peace and trade deals ending confl...,https://nypost.com/2025/10/26/us-news/trump-si...,69
176,2025-10-25-13-07-30 +0000,bbc,Trump hopes China will help push Russia toward...,https://www.bbc.com/news/articles/cn51yk9vgrko...,69
72,2025-10-26-01-51-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...,68
125,2025-10-25-20-30-41 +0000,nypost,Trump not meeting with Putin again until he kn...,https://nypost.com/2025/10/25/world-news/trump...,65


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
80,76,2025-10-26-01-00-00 +0000,wsj,"“You have zero zoning conditions, you’re the p...",https://www.wsj.com/politics/policy/trump-whit...
19,53,2025-10-26-09-47-45 +0000,nypost,Two men arrested over $100M Louvre Museum jewe...,https://nypost.com/2025/10/26/world-news/louvr...
94,38,2025-10-25-23-10-24 +0000,nypost,NYC voters run to the polls on first day of ea...,https://nypost.com/2025/10/25/us-news/nyc-vote...
105,37,2025-10-25-22-34-56 +0000,latimes,Arellano: He's just happy to root for the Dodg...,https://www.latimes.com/california/story/2025-...
50,33,2025-10-26-06-49-24 +0000,nypost,Trump signs peace and trade deals ending confl...,https://nypost.com/2025/10/26/us-news/trump-si...
54,31,2025-10-26-05-21-38 +0000,nypost,Bikini-clad Brazilian influencer Melissa Said ...,https://nypost.com/2025/10/26/world-news/brazi...
116,30,2025-10-25-21-25-09 +0000,nypost,NYC girl allegedly attacked by deranged stepda...,https://nypost.com/2025/10/25/us-news/nyc-girl...
166,29,2025-10-25-15-21-49 +0000,nypost,Chilling footage shows Russian drone hunting d...,https://nypost.com/2025/10/25/world-news/chill...
121,27,2025-10-25-20-55-57 +0000,nypost,Trump announces 10% tariff hike on Canada as f...,https://nypost.com/2025/10/25/us-news/trump-an...
176,26,2025-10-25-13-07-30 +0000,bbc,Trump hopes China will help push Russia toward...,https://www.bbc.com/news/articles/cn51yk9vgrko...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
